<a name="top"></a>
<img src="https://netacad.centralesupelec.fr/img/cs.jpg" width="200" style="display:inline;">

<h1>
<center>Airbnb price prediction</center>
</h1>

---


<center>
<nav>
<a href="https://github.com/flaviendeseure/Airbnb_berlin_price_prediction">Github</a> |
<a href="https://centralesupelec.edunao.com/course/view.php?id=5285">CentraleSupélec</a>
</nav>
</center>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../src/")
from utils import *
from coordinates_utils import distance_to_segment, is_point_close_segment, format_coords

from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.cluster import KMeans

from data import AccidentData
from format import merge_post_format

from geopy.distance import distance

import warnings 
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [2]:
acc_data = AccidentData()
df_merged = acc_data.preprocess_df(check=True)
df_merged.head()

[Check] Checking if data is in your computer...
[Check] Checking completed, no data is missing!
[Check] File already exists! loading file


,Num_Acc,grav,senc,catv,occutc,obs,obsm,choc,manv,num_veh,id_vehicule,motor,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1,vma,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,manv2,catv2,obs2,obsm2,choc2
0,200500000030,1.0,0.0,18,0.0,0.0,2.0,7.0,17.0,A01,NaN,NaN,3.0,37.0,0.0,NaN,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0,50,1.0,0.0,1.0,0.0,NaN,5,1,13,1945,5,2,1,1.0,3.0,331.0,rue de la chapelle,M,5030000.0,284000.0,620,11.0,1.0,0.0,2.0,8.0
1,200500000034,2.0,0.0,10,0.0,0.0,9.0,3.0,15.0,A01,NaN,NaN,3.0,127.0,0.0,NaN,2.0,0.0,0.0,0.0,0.0,1.0,3.0,0,50,1.0,0.0,1.0,0.0,NaN,5,1,19,1045,1,1,1,7.0,1.0,22.0,NaN,M,0.0,0.0,620,1.0,1.0,0.0,2.0,1.0
2,200500000078,1.0,0.0,7,0.0,0.0,2.0,2.0,15.0,A01,NaN,NaN,3.0,56.0,0.0,NaN,2.0,2.0,15.0,100.0,0.0,2.0,1.0,0,0,1.0,0.0,1.0,3.0,NaN,5,1,26,1315,1,1,9,1.0,3.0,173.0,NaN,M,0.0,0.0,20,1.0,1.0,0.0,2.0,1.0
3,200500000093,2.0,0.0,7,0.0,0.0,2.0,3.0,21.0,A01,NaN,NaN,3.0,81.0,0.0,NaN,2.0,0.0,0.0,0.0,0.0,1.0,2.0,0,52,1.0,0.0,1.0,0.0,NaN,5,1,3,1330,1,2,1,1.0,1.0,810.0,rue du grand montoir,M,4925500.0,309400.0,20,21.0,1.0,0.0,2.0,3.0
4,200500000170,1.0,0.0,1,0.0,0.0,2.0,4.0,2.0,A01,NaN,NaN,3.0,11.0,0.0,NaN,2.0,2.0,27.0,200.0,0.0,1.0,1.0,0,50,1.0,0.0,1.0,0.0,NaN,5,1,29,1830,3,1,1,1.0,2.0,196.0,NaN,M,0.0,0.0,760,2.0,10.0,0.0,2.0,2.0


In [3]:
df_format = merge_post_format(df_merged)
df_format = df_format.replace(pd.NA, np.nan)
df_format.head()

,grav,senc,catv,occutc,obs,obsm,choc,num_veh,id_vehicule,motor,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,surf,infra,situ,env1,hrmn,lum,int,atm,col,com,adr,gps,lat,long,dep,obs2,obsm2,choc2,date,is_in_agg
0,1,0.0,3.0,0.0,0.0,2.0,2.0,A01,NaN,NaN,4,0.0,NaN,NaN,2.0,NaN,NaN,NaN,3.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1700,1,0.0,1.0,6.0,101.0,PONT DU CARROUSEL,NaN,NaN,NaN,750,0.0,2.0,2.0,2009-04-30,1
1,2,0.0,3.0,0.0,0.0,2.0,1.0,B01,NaN,NaN,4,0.0,NaN,NaN,2.0,NaN,NaN,NaN,0.0,1.0,1.0,20.0,1.0,0.0,1.0,3.0,1345,1,0.0,1.0,3.0,201.0,"20 bis, AEROPORT (ROUTE",NaN,NaN,NaN,470,0.0,2.0,8.0,2009-03-23,1
2,2,0.0,3.0,0.0,0.0,9.0,1.0,B02,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,1130,1,2.0,1.0,1.0,242.0,R CHAPELLE/PAS.ECOLIERS,M,0.0,0,740,0.0,2.0,8.0,2009-03-21,1
3,2,0.0,2.0,0.0,0.0,2.0,1.0,B01,NaN,NaN,4,0.0,NaN,NaN,2.0,NaN,NaN,NaN,3.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,2030,1,0.0,1.0,6.0,110.0,"118, BOULEVARD DE MAGENT",NaN,NaN,NaN,750,8.0,2.0,1.0,2009-06-03,1
4,1,0.0,NaN,0.0,14.0,0.0,1.0,A01,NaN,NaN,4,0.0,NaN,NaN,3.0,NaN,NaN,NaN,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,305,3,0.0,1.0,6.0,113.0,"5bis, BOULEVARD DE L HOP",NaN,NaN,NaN,750,NaN,NaN,NaN,2009-06-05,1


In [13]:
missing(df_format)

lum => no missing values [0%]
int => 2 [0.0%]
atm => 6 [0.01%]
col => 302 [0.38%]
catr => no missing values [0%]
circ => 1067 [1.35%]
nbv => 379 [0.48%]
vosp => 442 [0.56%]
prof => 207 [0.26%]
plan => 225 [0.28%]
surf => 203 [0.26%]
infra => 1253 [1.58%]
situ => 519 [0.66%]
catv => 11108 [14.03%]
obs => 64 [0.08%]
obsm => 53 [0.07%]
choc => 38 [0.05%]
grav => no missing values [0%]
obs2 => 11113 [14.04%]
obsm2 => 11105 [14.03%]
choc2 => 11081 [14.0%]
date => no missing values [0%]
is_in_agg => no missing values [0%]


In [11]:
imp = KNNImputer(missing_values = np.nan, n_neighbors=1)
series_date = df_format[["date"]]
df_num = df_format.drop('date', axis=1)
df_not_missing = pd.DataFrame(imp.fit_transform(df_num), columns=df_num.columns)
df_not_missing_date = pd.concat([df_not_missing, series_date])
df_not_missing_date.head()

,lum,int,atm,col,catr,circ,nbv,vosp,prof,plan,surf,infra,situ,catv,obs,obsm,choc,grav,obs2,obsm2,choc2,is_in_agg,date
0,1.0,0.0,1.0,6.0,4.0,2.0,4.0,3.0,1.0,1.0,1.0,0.0,1.0,3.0,0.0,2.0,2.0,1.0,0.0,2.0,2.0,1.0,NaN
1,1.0,0.0,1.0,3.0,4.0,2.0,3.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,0.0,2.0,1.0,2.0,0.0,2.0,8.0,1.0,NaN
2,1.0,2.0,1.0,1.0,4.0,2.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,3.0,0.0,9.0,1.0,2.0,0.0,2.0,8.0,1.0,NaN
3,1.0,0.0,1.0,6.0,4.0,2.0,3.0,3.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,2.0,1.0,2.0,8.0,2.0,1.0,1.0,NaN
4,3.0,0.0,1.0,6.0,4.0,3.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,3.0,14.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,NaN


In [12]:
df_encoded = one_hot_encoder(df_not_missing, ['int', 'col', 'catr', 'vosp','prof', 'plan', 'surf', 
                                              'infra', 'situ', 'catv', 'obs', 'obsm', 'choc', 'obs2', 'obsm2', 'choc2'])
df_encoded.head()                             

,lum,atm,circ,nbv,grav,is_in_agg,int_0.0,int_1.0,int_2.0,int_3.0,int_4.0,int_5.0,int_6.0,int_7.0,int_8.0,col_1.0,col_2.0,col_3.0,col_4.0,col_5.0,col_6.0,col_7.0,catr_1.0,catr_2.0,catr_3.0,catr_4.0,catr_5.0,catr_6.0,catr_7.0,catr_9.0,vosp_0.0,vosp_1.0,vosp_2.0,vosp_3.0,prof_0.0,prof_1.0,prof_2.0,prof_3.0,prof_4.0,plan_0.0,plan_1.0,plan_2.0,plan_3.0,plan_4.0,surf_0.0,surf_1.0,surf_2.0,surf_3.0,surf_4.0,surf_5.0,infra_0.0,infra_1.0,infra_2.0,infra_3.0,infra_4.0,infra_5.0,infra_6.0,infra_7.0,infra_8.0,infra_9.0,situ_0.0,situ_1.0,situ_2.0,situ_3.0,situ_4.0,situ_5.0,situ_6.0,situ_8.0,catv_1.0,catv_2.0,catv_3.0,catv_4.0,catv_5.0,catv_6.0,catv_7.0,obs_0.0,obs_1.0,obs_2.0,obs_3.0,obs_4.0,obs_5.0,obs_6.0,obs_7.0,obs_8.0,obs_9.0,obs_10.0,obs_11.0,obs_12.0,obs_13.0,obs_14.0,obs_15.0,obs_16.0,obs_17.0,obsm_0.0,obsm_1.0,obsm_2.0,obsm_4.0,obsm_5.0,obsm_6.0,obsm_9.0,choc_0.0,choc_1.0,choc_2.0,choc_3.0,choc_4.0,choc_5.0,choc_6.0,choc_7.0,choc_8.0,choc_9.0,obs2_0.0,obs2_1.0,obs2_2.0,obs2_3.0,obs2_4.0,obs2_5.0,obs2_6.0,obs2_7.0,obs2_8.0,obs2_9.0,obs2_10.0,obs2_11.0,obs2_12.0,obs2_13.0,obs2_14.0,obs2_15.0,obs2_16.0,obsm2_0.0,obsm2_1.0,obsm2_2.0,obsm2_4.0,obsm2_5.0,obsm2_6.0,obsm2_9.0,choc2_0.0,choc2_1.0,choc2_2.0,choc2_3.0,choc2_4.0,choc2_5.0,choc2_6.0,choc2_7.0,choc2_8.0,choc2_9.0
0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,2.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,1.0,2.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,1.0,3.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [18]:
km = KMeans()
x1 = km.fit(df_encoded)

In [19]:
x1.labels_

array([7, 4, 6, ..., 3, 2, 0])

1. One Hot encoding
- vosp: Voie spéciale
- prof: Profil
- plan: Plan
- surf: Surface
- infra: Infrastructure
- situ: Situation
- catr: Catégorie de route
- int: Intersection
- col: Collision
- obs: Obstacle fixe heurté
- obsm: Obstacle mobile heurté
- choc: Point de choc initial
- catv: Catégorie de véhicule

2. Categorical encoding
- grav: Gravité de l'accident
- nbv: Nombre de voies
- circ: Circulation
- agg: Localisation (deja fait)
- lum: Conditions d'éclairage dans lesquelles l'accident s'est produit
- atm: Conditions Atmosphère

3. Date
- date: date



In [3]:
import geopandas as gpd
df = gpd.read_file(os.path.join("..", "data", "france-20230101.geojson"))
df_lat_long = create_lat_long(df)
df_lat_long_preprocess = preprocessing(df_lat_long)
df_lat_long_preprocess.head()

,code_com,ame,regime,date_maj,latitude_dep,longitude_dep,latitude_fin,longitude_fin
0,86274,AUTRE,EN AGGLOMERATION,2020-07-28,47.086212,0.005851,47.086436,0.005750
1,47165,PISTE CYCLABLE,AUTRE,2018-08-20,44.529839,0.024023,44.529895,0.024091
2,65440,VOIE VERTE,AUTRE,2015-09-16,43.220046,0.047086,43.220395,0.046962
3,61001,AMENAGEMENT MIXTE PIETON VELO HORS VOIE VERTE,AUTRE,2021-12-23,48.447847,0.087109,48.447845,0.087168
4,65047,BANDE CYCLABLE,EN AGGLOMERATION,2021-08-03,43.241280,0.094978,43.241165,0.094934


In [4]:
df_merged["lat"] = df_merged["lat"].apply(lambda x: str(x).replace(",", ".")).astype(float)
df_merged["long"] = df_merged["long"].apply(format_coords).astype("Float64")

In [5]:
# 39Lat: valencia, 52Lat: Londres, -8.5Long: Porto, 9.5Long: Milan
df_coords_cleaned = df_merged[(df_merged["lat"]>39) & (df_merged["lat"]<52) & (df_merged["long"]>-8.5) & (df_merged["long"]<9.5)]
print(df_coords_cleaned.shape)
print(df_coords_cleaned.lat.isna().sum(), df_coords_cleaned.long.isna().sum())
df_coords_cleaned.head()    

(14420, 50)
0 0


,Num_Acc,grav,senc,catv,occutc,obs,obsm,choc,manv,num_veh,id_vehicule,motor,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1,vma,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,manv2,catv2,obs2,obsm2,choc2
65876,201900000037,1.0,2.0,1,NaN,0.0,2.0,8.0,19.0,B01,138Â 306Â 460,5.0,4.0,JEAN JAURES (AVENUE),0.0,NaN,3.0,5.0,0,0,2.0,1.0,1.0,NaN,NaN,2.0,4.0,6.0,NaN,50.0,2019,11,29,13:40,1,2,1,2.0,3.0,67482,JEAN JAURES (AVENUE),NaN,48.570848,7.758768,67,1.0,40.0,0.0,2.0,0.0
65877,201900000038,1.0,2.0,1,NaN,0.0,2.0,1.0,1.0,B01,138Â 306Â 458,5.0,4.0,MOULIN (PLACE JEAN),0.0,NaN,2.0,2.0,0,0,0.0,1.0,1.0,NaN,NaN,1.0,0.0,1.0,NaN,30.0,2019,11,29,13:50,1,2,1,1.0,6.0,66136,MOULIN (PLACE JEAN),NaN,42.696950,2.89935,66,15.0,7.0,0.0,2.0,1.0
65878,201900000049,1.0,1.0,1,NaN,14.0,0.0,0.0,0.0,A01,138Â 306Â 442,5.0,4.0,TURPIN DE CRISSE AVENUE,0.0,NaN,2.0,2.0,0,0,0.0,1.0,1.0,NaN,NaN,1.0,0.0,1.0,NaN,60.0,2019,11,28,08:16,1,2,1,1.0,7.0,49007,TURPIN DE CRISSE AVENUE,NaN,47.465340,-0.55433,49,NaN,NaN,NaN,NaN,NaN
65879,201900000051,1.0,0.0,1,NaN,0.0,2.0,0.0,9.0,B01,138Â 306Â 437,5.0,4.0,RUE DES MONTEES,0.0,NaN,1.0,2.0,(1),(1),0.0,1.0,2.0,NaN,NaN,1.0,0.0,1.0,NaN,50.0,2019,11,29,17:10,2,2,6,1.0,3.0,45234,RUE DES MONTEES,NaN,47.876380,1.9145,45,9.0,7.0,0.0,2.0,0.0
65880,201900000059,1.0,0.0,1,NaN,0.0,2.0,1.0,1.0,B01,138Â 306Â 424,5.0,4.0,CRS JOURNU AUBER,0.0,NaN,1.0,2.0,(1),(1),1.0,1.0,1.0,NaN,NaN,2.0,0.0,1.0,NaN,50.0,2019,11,22,18:45,5,2,2,2.0,3.0,33063,CRS JOURNU AUBER,NaN,44.861650,-0.573,33,1.0,7.0,0.0,2.0,1.0


In [44]:
from geopy.geocoders import Nominatim
import requests

def get_communeName_postalCode(insee_code):
    url = f"https://geo.api.gouv.fr/communes/{insee_code}?fields=nom&format=json&geometry=centre"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()["nom"]
    return None

def get_gps_from_address(row):
    geolocator = Nominatim(user_agent="myGeocoder")
    code_com = str(int(float(row["com"])))
    code_len = len(code_com)
    print(code_com, code_len)
    if code_len < 5 :
        code_com = code_com if code_len == 3 else "0" + code_com if code_len == 2 else "00" + code_com
        code_insee = str(row["dep"]) + code_com
    print(row["dep"])
    commune_name = get_communeName_postalCode(code_insee)
    if commune_name is not None:
        adr = row["adr"] + ", " + commune_name
        print(adr)
    location = geolocator.geocode(row["adr"])
    return location.latitude, location.longitude

get_gps_from_address(df_merged[df_merged["lat"].isna()].iloc[-1,:])

416 3
974


AttributeError: 'NoneType' object has no attribute 'latitude'

In [40]:
int(float('416.0'))

416

In [ ]:
(50.8415576, 4.3423164)

In [22]:
df_merged[df_merged["lat"].isna()].loc[:,["an","lat","long","com","adr","dep"]]

,an,lat,long,com,adr,dep
61,5,NaN,<NA>,691.0,"147, PARIS DE 1 A 187/21",20
62,5,NaN,<NA>,101.0,PENSEES ( RUE DES ),30
63,5,NaN,<NA>,29.0,MIDI (BOULEVARD DU),60
64,5,NaN,<NA>,88.0,"0, COMBOUL (AV. RAYMOND)",60
65,5,NaN,<NA>,88.0,"20, PERTINAX (R.)",60
...,...,...,...,...,...,...
65782,18,NaN,<NA>,101.0,"129, PETIT PEROU",971
65783,18,NaN,<NA>,101.0,"5, RN5-BOISRIPEAUX",971
65828,18,NaN,<NA>,407.0,ROUTE DU SACRE COEUR,974
65847,18,NaN,<NA>,411.0,"33, VIGIE (CHEMIN DE LA)",974


In [19]:
df_merged[df_merged["lat"].isna()].loc[63,["lat","long","com","adr"]]

lat                     NaN
long                   <NA>
com                    29.0
adr     MIDI (BOULEVARD DU)
Name: 63, dtype: object

In [6]:
#Importing the Nominatim geocoder class 
from geopy.geocoders import Nominatim
 
#address we need to geocode
loc = 'Taj Mahal, Agra, Uttar Pradesh 282001'
 
#making an instance of Nominatim class
geolocator = Nominatim(user_agent="my_request")
 
#applying geocode method to get the location
location = geolocator.geocode(loc)
 
#printing address and coordinates
print((location.latitude, location.longitude))

(27.1682576, 78.0507466)


In [93]:
df_lat_long_preprocess.head()

,code_com,ame,regime,date_maj,latitude_dep,longitude_dep,latitude_fin,longitude_fin
0,86274,AUTRE,EN AGGLOMERATION,2020-07-28,47.086212,0.005851,47.086436,0.005750
1,47165,PISTE CYCLABLE,AUTRE,2018-08-20,44.529839,0.024023,44.529895,0.024091
2,65440,VOIE VERTE,AUTRE,2015-09-16,43.220046,0.047086,43.220395,0.046962
3,61001,AMENAGEMENT MIXTE PIETON VELO HORS VOIE VERTE,AUTRE,2021-12-23,48.447847,0.087109,48.447845,0.087168
4,65047,BANDE CYCLABLE,EN AGGLOMERATION,2021-08-03,43.241280,0.094978,43.241165,0.094934


In [ ]:
df_coords_cleaned["id_segment"] = np.nan
# iterate over all the rows in the dataframe df_coords_cleaned
for index, row in df_coords_cleaned.iterrows():
    # apply the function is_point_close_segment to each row
    row["id_segment"] = df_lat_long_preprocess.apply(
                            lambda x: is_point_close_segment(
                                (row['lat'], row['long']), 
                                (x['latitude_dep'], x['longitude_dep']), 
                                (x['latitude_fin'], x['longitude_fin'])), 
                            axis=1)

In [109]:
is_point_close_segment((47.086936,	0.005790), 
        (df_lat_long_preprocess.loc[0,'latitude_dep'], df_lat_long_preprocess.loc[0,'longitude_dep']), 
        (df_lat_long_preprocess.loc[0,'latitude_fin'], df_lat_long_preprocess.loc[0,'longitude_fin']))


False

In [31]:
.shape

(46253, 2)

In [29]:
df_merged.long.dropna().shape

(46253,)

In [41]:
df_merged[df_merged["lat"].isna()].adr.isna().sum()

61

In [32]:
df_merged[["lat","long"]].dropna()[(df_merged[["lat","long"]].dropna()["lat"]!=0.0) & (df_merged[["lat","long"]].dropna()["long"]!=0.0)]

,lat,long
0,5.030000e+06,284000.0
3,4.925500e+06,309400.0
6,4.859000e+06,-230000.0
10,4.748500e+06,-246700.0
14,4.908600e+06,-109200.0
...,...,...
80846,4.881049e+01,2.36194
80847,4.323095e+01,-0.276584
80848,4.868499e+01,6.176019
80849,4.887691e+01,2.366594


In [ ]:
df_merged[["lat","long"]]

In [ ]:
def f

In [12]:
id = None
for index, row in df_lat_long_preprocess.iterrows():
    print(f"{index}/{df_lat_long_preprocess.shape[0]}", end="\r")
    #print(row['latitude_dep'], row['longitude_dep'], row['latitude_fin'], row['longitude_fin'])
    if is_point_close_segment((47.086936,0.005790), 
        (row['latitude_dep'], row['longitude_dep']), 
        (row['latitude_fin'], row['longitude_fin'])):
        id = index
        break

In [20]:
for row in df_lat_long_preprocess.itertuples():
    if is_point_close_segment((47.086936,0.005790), 
        (row.latitude_dep, row.longitude_dep), 
        (row.latitude_fin, row.longitude_fin)):
        id = row.Index
        break

In [11]:
list_index

[166246]

In [102]:
df_lat_long_preprocess.apply(
    lambda x: is_point_close_segment(
        (47.086936,	0.005790), 
        (x['latitude_dep'], x['longitude_dep']), 
        (x['latitude_fin'], x['longitude_fin'])), 
    axis=1)

KeyboardInterrupt: 

In [16]:
is_point_close_segment()

count    273061.000000
mean         23.946650
std          44.404432
min           0.000006
25%           5.782106
50%          10.631778
75%          24.739565
max        2504.428193
Name: distance, dtype: float64

In [41]:
df_merged[df_merged.lat==0]

,Num_Acc,grav,senc,catv,occutc,obs,obsm,choc,manv,num_veh,id_vehicule,motor,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1,vma,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,manv2,catv2,obs2,obsm2,choc2
1,200500000034,2.0,0.0,10,0.0,0.0,9.0,3.0,15.0,A01,NaN,NaN,3.0,127.0,0.0,NaN,2.0,0.0,0.0,0.0,0.0,1.0,3.0,0,50,1.0,0.0,1.0,0.0,NaN,5,1,19,1045,1,1,1,7.0,1.0,22.0,NaN,M,0.0,0.0,620,1.0,1,0.0,2.0,1.0
2,200500000078,1.0,0.0,7,0.0,0.0,2.0,2.0,15.0,A01,NaN,NaN,3.0,56.0,0.0,NaN,2.0,2.0,15.0,100.0,0.0,2.0,1.0,0,0,1.0,0.0,1.0,3.0,NaN,5,1,26,1315,1,1,9,1.0,3.0,173.0,NaN,M,0.0,0.0,20,1.0,1,0.0,2.0,1.0
4,200500000170,1.0,0.0,1,0.0,0.0,2.0,4.0,2.0,A01,NaN,NaN,3.0,11.0,0.0,NaN,2.0,2.0,27.0,200.0,0.0,1.0,1.0,0,50,1.0,0.0,1.0,0.0,NaN,5,1,29,1830,3,1,1,1.0,2.0,196.0,NaN,M,0.0,0.0,760,2.0,10,0.0,2.0,2.0
5,200500000246,2.0,0.0,7,0.0,0.0,0.0,6.0,0.0,A01,NaN,NaN,3.0,92.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,60,1.0,5.0,1.0,0.0,NaN,5,1,23,1800,1,2,6,1.0,1.0,207.0,rond point ZA noyal sud,M,0.0,0.0,350,1.0,1,0.0,2.0,7.0
7,200500000273,1.0,0.0,7,0.0,0.0,2.0,7.0,1.0,A01,NaN,NaN,4.0,0.0,0.0,NaN,2.0,2.0,0.0,0.0,0.0,1.0,1.0,0,60,1.0,0.0,1.0,0.0,NaN,5,1,5,1730,1,2,1,1.0,3.0,69.0,NaN,M,0.0,0.0,290,9.0,1,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56305,201600058371,1.0,1.0,1,0.0,16.0,0.0,8.0,13.0,A01,NaN,NaN,2.0,1,NaN,NaN,3.0,6.0,1.0,150.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,99.0,NaN,16,3,20,1155,1,2,1,1.0,7.0,209.0,"2, GENERAL DE GAULLE (BL",A,0.0,0.0,972,None,None,None,None,None
56306,201600058382,1.0,1.0,1,0.0,0.0,2.0,1.0,1.0,B01,NaN,NaN,4.0,NaN,NaN,NaN,2.0,2.0,NaN,NaN,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,99.0,NaN,16,6,3,1805,1,2,1,1.0,6.0,209.0,"4, CENTRE (BOULEVARD DU)",A,0.0,0.0,972,2.0,32,0.0,2.0,2.0
56307,201600058410,1.0,1.0,7,0.0,0.0,2.0,7.0,1.0,B01,NaN,NaN,3.0,40,NaN,NaN,2.0,2.0,0.0,100.0,0.0,1.0,1.0,NaN,0.0,1.0,0.0,1.0,99.0,NaN,16,8,25,1255,1,1,2,1.0,6.0,209.0,"3, SOLEIL LEVANT (BOULEV",A,0.0,0.0,972,13.0,1,0.0,2.0,1.0
56313,201600058537,2.0,0.0,1,0.0,0.0,0.0,0.0,0.0,Z01,NaN,NaN,4.0,0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,80.0,1.0,0.0,1.0,0.0,NaN,16,1,13,1958,2,2,3,1.0,2.0,302.0,"1, CABASSOU (CITE)",G,0.0,0.0,973,0.0,2,0.0,9.0,1.0


In [39]:
df_merged.query("lat > 1000")

TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
accidents.query("distance <= @radius")

In [7]:
import pandas as pd
import numpy as np

def link_dataframes(accidents, infrastructure, radius):
    distance_a_verifier = distance_to_segment(coordonnee_a_verifier, coordonnee_debut_route, coordonnee_fin_route)
    if distance_a_verifier <= 20:

    for i, row in infrastructure.iterrows():
        
        lat1, lon1 = row['lat'], row['lon']
        accidents['distance'] = distance((lat1, lon1), (accidents['lat'], accidents['lon'])).m
        infrastructure.at[i, 'distance'] = accidents.query("distance <= @radius").distance.min()
    # On joint les deux dataframes
    result = accidents.merge(infrastructure, on=["distance"], how="inner")
    result = result[result.date_accident > result.date_mise_en_service_infrastructure]
    return result

# On utilise les dataframes accidents et infrastructure
result = link_dataframes(df_merged, df_lat_long_preprocess, 20)

KeyError: 'lat'